In [1]:
# https://medium.com/@HeeebsInc/using-machine-learning-to-predict-daily-fantasy-basketball-scores-part-i-811de3c54a98

In [1]:
# needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://www.basketball-reference.com/playoffs/"

# collect HTML data
html = urlopen(url)
        
# create beautiful soup object from HTML
soup = BeautifulSoup(html, features="lxml")

# use getText()to extract the headers into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

# get rows from table
rows = soup.findAll('tr')[2:]
rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

# if you print row_data here you'll see an empty row
# so, remove the empty row
rows_data.pop(20)
# for simplicity subset the data for only 39 seasons
rows_data = rows_data[0:38]

# we're missing a column for years
# add the years into rows_data
last_year = 2020
for i in range(0, len(rows_data)):
    rows_data[i].insert(0, last_year)
    last_year -=1
    
# create the dataframe
nba_finals = pd.DataFrame(rows_data, columns = headers)
# export dataframe to a CSV 
nba_finals.to_csv("nba_finals_history.csv", index=False)

# Web Scraper

In [2]:
# https://github.com/klaudius1D/ShootersShoot/tree/main

In [15]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup

# settings
pd.set_option('display.max_columns', None)

In [4]:
# Set Playoff Years from 2020 to 2023
years = list(range(2020, 2024))

# {} used to vary dates in loop
url_start = "https://www.basketball-reference.com/playoffs/NBA_{}_per_game.html"

# Replace brackets with specific years based on list
for year in years:
    url = url_start.format(year)
    data = requests.get(url)

    # Create HTML files for each year
    with open("topPicksHTML/Playoffs-{}_topPicks.html".format(year), "wb") as f:
        f.write(data.content)

In [5]:
# using a loop, parse each html for each year in years list
dfs = []
for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    soup = BeautifulSoup(data.content, "html.parser")
    topPicks_table = soup.find("table", {"id": "per_game_stats"})
    topPicks_df = pd.read_html(str(topPicks_table))[0]
    topPicks_df = topPicks_df[topPicks_df.Rk != "Rk"]
    topPicks_df["Year"] = year

    dfs.append(topPicks_df)

    # Save all contents into one csv files into new folder called topPicksCSV
    topPicks_df.to_csv("topPicksCSV/Playoffs-{}_topPicks.csv".format(year), index=False)

In [6]:
# combine all dataframes into one
df = pd.concat(dfs)
df.to_csv("topPicksCSV/Playoffs-Master_topPicks.csv", index=False)

In [7]:
# check contents of dataframe, ensure all years are included
df.head()
df.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
222,213,Ziaire Williams,SF,21,MEM,4,0,3.0,0.5,1.8,...,0.0,0.5,0.5,0.5,0.0,0.0,0.5,0.3,1.3,2023
223,214,Trae Young,PG,24,ATL,6,6,38.3,10.0,24.8,...,0.8,2.8,3.7,10.2,1.7,0.7,4.0,1.8,29.2,2023
224,215,Omer Yurtseven,C,24,MIA,8,0,2.0,0.3,0.9,...,0.4,0.3,0.6,0.1,0.0,0.1,0.1,0.3,0.5,2023
225,216,Cody Zeller,C,30,MIA,21,0,8.3,1.0,1.7,...,0.5,1.8,2.3,0.3,0.1,0.2,0.6,1.3,2.2,2023
226,217,Ivica Zubac,C,25,LAC,5,5,26.0,3.4,6.0,...,3.2,6.4,9.6,0.6,0.6,0.2,2.2,1.6,9.2,2023


# Testing

In [16]:
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Jaylen Adams,PG,23,POR,3,0,7.0,1.0,3.0,.333,0.0,1.3,.000,1.0,1.7,.600,.333,0.0,0.0,NaN,0.3,0.7,1.0,0.7,0.3,0.0,0.0,0.3,2.0,2020
1,2,Steven Adams,C,26,OKC,7,7,30.0,4.4,7.4,.596,0.0,0.1,.000,4.4,7.3,.608,.596,1.3,2.9,.450,4.9,6.7,11.6,1.3,0.6,0.3,2.3,1.4,10.1,2020
2,3,Bam Adebayo,PF,22,MIA,19,19,36.2,6.5,11.6,.564,0.0,0.1,.000,6.5,11.5,.566,.564,4.7,6.1,.783,2.5,7.8,10.3,4.4,1.0,0.8,2.5,3.1,17.8,2020
3,4,Jarrett Allen,C,21,BRK,4,4,33.0,3.5,6.0,.583,0.0,0.0,NaN,3.5,6.0,.583,.583,3.3,4.0,.813,4.3,10.5,14.8,2.3,0.5,1.8,2.8,1.8,10.3,2020
4,5,Justin Anderson,SG,26,BRK,3,0,9.3,1.7,4.0,.417,1.7,3.7,.455,0.0,0.3,.000,.625,1.3,1.3,1.000,0.0,2.7,2.7,1.0,0.0,0.3,0.7,0.7,6.3,2020


In [30]:
df.info()

# NEED TO CONVERT THESE COLUMNS TO INT

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 226
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      890 non-null    object
 1   Player  890 non-null    object
 2   Pos     890 non-null    object
 3   Age     890 non-null    object
 4   Tm      890 non-null    object
 5   G       890 non-null    object
 6   GS      890 non-null    object
 7   MP      890 non-null    object
 8   FG      890 non-null    object
 9   FGA     890 non-null    object
 10  FG%     851 non-null    object
 11  3P      890 non-null    object
 12  3PA     890 non-null    object
 13  3P%     746 non-null    object
 14  2P      890 non-null    object
 15  2PA     890 non-null    object
 16  2P%     822 non-null    object
 17  eFG%    851 non-null    object
 18  FT      890 non-null    object
 19  FTA     890 non-null    object
 20  FT%     695 non-null    object
 21  ORB     890 non-null    object
 22  DRB     890 non-null    ob

In [20]:
clean_per_30 = [
    "Player", 
    "MP", 
    "FG%",
    "FT%",
    "3P",
    "PTS",
    "TRB", 
    "AST",
    "STL",
    "BLK",
    "TOV",

]

In [21]:
per_30_stats = df[clean_per_30]

In [28]:
per_30_stats[["PTS", 
             "TRB", 
             "AST",
             "STL",
             "BLK",
             "TOV"]] = df[["PTS",
    "TRB", 
    "AST",
    "STL",
    "BLK",
    "TOV"]].div(df['MP'], axis = 0)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [22]:
per_30_stats

,Player,MP,FG%,FT%,3P,PTS,TRB,AST,STL,BLK,TOV
0,Jaylen Adams,7.0,.333,NaN,0.0,2.0,1.0,0.7,0.3,0.0,0.0
1,Steven Adams,30.0,.596,.450,0.0,10.1,11.6,1.3,0.6,0.3,2.3
2,Bam Adebayo,36.2,.564,.783,0.0,17.8,10.3,4.4,1.0,0.8,2.5
3,Jarrett Allen,33.0,.583,.813,0.0,10.3,14.8,2.3,0.5,1.8,2.8
4,Justin Anderson,9.3,.417,1.000,1.7,6.3,2.7,1.0,0.0,0.3,0.7
...,...,...,...,...,...,...,...,...,...,...,...
222,Ziaire Williams,3.0,.286,NaN,0.3,1.3,0.5,0.5,0.0,0.0,0.5
223,Trae Young,38.3,.403,.860,3.0,29.2,3.7,10.2,1.7,0.7,4.0
224,Omer Yurtseven,2.0,.286,NaN,0.0,0.5,0.6,0.1,0.0,0.1,0.1
225,Cody Zeller,8.3,.571,.400,0.0,2.2,2.3,0.3,0.1,0.2,0.6


# Data Cleaning

In [1]:
import pandas as pd
import numpy as np

# create variables for scoring settings
global pointsScored, totalRebounds, assists, steals, blockedShots, turnovers
pointsScored = 1.0
totalRebounds = 1.2
assists = 1.5
steals = 3.0
blockedShots = 3.0
turnovers = -1.0

In [2]:
# load Master_topPicks into dataframe
df = pd.read_csv("topPicksCSV/Playoffs-Master_topPicks.csv")

# keep only the columns we need
stats_to_keep = [
    "Player",
    "Pos",
    "Tm",
    "G",
    "MP",
    "TRB",
    "AST",
    "STL",
    "BLK",
    "TOV",
    "PTS",
    "Year",
]
df = df[stats_to_keep]

# capture each player occurences
# df = df.groupby(["Player"]).sum().reset_index()

# if a player has played for multiple years, average their stats across those years
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns
df = df.groupby(["Player"])[numeric_cols].mean().reset_index()

# drop year column
df = df.drop("Year", axis=1)

In [3]:
# create calculated column for fantasy points
df["FantasyPoints"] = (
    df["PTS"] * pointsScored
    + df["TRB"] * totalRebounds
    + df["AST"] * assists
    + df["STL"] * steals
    + df["BLK"] * blockedShots
    + df["TOV"] * turnovers)

#round to the nearest decimal
df = df.round(1)

In [4]:
# write to csv
df.to_csv("topPicksCSV/Playoffs-Master-Combined_topPicks.csv", index=False)

df

,Player,G,MP,TRB,AST,STL,BLK,TOV,PTS,FantasyPoints
0,Aaron Gordon,11.7,32.5,6.2,2.4,0.5,0.7,1.3,12.7,26.1
1,Aaron Holiday,3.7,8.4,0.6,1.7,0.5,0.1,0.7,3.8,8.0
2,Aaron Nesmith,10.0,9.2,1.8,0.2,0.2,0.2,0.2,2.0,5.4
3,Abdel Nader,4.0,7.0,1.0,0.0,0.2,0.4,0.0,1.0,3.8
4,Al Horford,15.7,32.9,7.9,2.9,0.7,1.4,0.9,8.6,28.0
...,...,...,...,...,...,...,...,...,...,...
455,Yogi Ferrell,9.0,1.7,0.3,0.3,0.0,0.0,0.0,0.4,1.2
456,Yuta Watanabe,2.5,3.8,0.5,0.0,0.0,0.0,0.6,2.0,2.0
457,Zach LaVine,4.0,38.3,5.3,6.0,0.8,0.3,3.0,19.3,35.0
458,Zeke Nnaji,4.0,3.5,0.3,0.1,0.1,0.0,0.1,1.6,2.5
